### STEP 1 | Input Spectrum Classification

This script loads spectral data from files, processes and normalizes it, and then trains a convolutional neural network (CNN) to analyze the spectra. The CNN is used to find the closest matches between test and training data based on Euclidean distances of encoded representations. The script also generates various plots to visualize the predictions and reconstructions, saving them in a PDF. Additionally, it processes an input spectrum, compares it with the trained data, and outputs the closest match and CNN reconstruction.

**Inputs:**
- *Data directories* 
  - C:\\1. AI - ITACA\\9. JupyterNotebook\\JPL_CH3CN
  - C:\1. AI - ITACA\9. JupyterNotebook\JPL_CO
  - C:\1. AI - ITACA\9. JupyterNotebook\JPL_HCO+v=0,1,2
  - C:\1. AI - ITACA\9. JupyterNotebook\JPL_SiO
  - C:\1. AI - ITACA\9. JupyterNotebook\ALL
- *Input spectrum file* 
  - "C:\\1. AI - ITACA\\9. JupyterNotebook\\_BK\\example_4mols_format"
- *CNN model input shape*  
- *Test split ratio* → `test_size=0.2`  
- *Interpolation points* → `interp_points=1000`

**Outputs:**
- *Interpolated and normalized data* 
- *Trained CNN model* 
- *Encoded representations* → `encoded_train`, `encoded_test`  
- *Closest matches* 
- *Reconstructed spectra* → `reconstructed_test`, `reconstructed`  
- *Visualization plots*
  - Predicted LogN vs True LogN
  - Predicted Tex vs True Tex
  - 10 sandom spectra from CNN dataset
  - Input Spectrum Classification
- *PDF report*
  - "CNN_PERFORMANCE_+_INPUTSPECCLASS_JPL_CH3CN.pdf"

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
import tensorflow as tf
from tensorflow.keras import layers
import re
from matplotlib.backends.backend_pdf import PdfPages
import traceback

# 1. DATA LOADING FUNCTION
def load_data(folder, interp_points=30000):
    spectra, frequencies, headers, filenames_list, logn_values, tex_values = [], [], [], [], [], []
    
    for filename in os.listdir(folder):
        if "_simulate_generate" in filename and "_cleaned_simulate_generate" not in filename:
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as file:
                lines = file.readlines()
                if len(lines) < 3:
                    continue
                
                header = lines[0].strip()
                logn_match = re.search(r'logn=([0-9]+\.?[0-9]*)', header)
                tex_match = re.search(r'tex=([0-9]+\.?[0-9]*)', header)
                
                if logn_match and tex_match:
                    logn_value = float(logn_match.group(1))
                    tex_value = float(tex_match.group(1))
                else:
                    continue
                
                data_lines = lines[2:]
                try:
                    data = np.array([list(map(float, line.split())) for line in data_lines])
                    freq, spec = data[:, 0], data[:, 1]
                    frequencies.append(freq)
                    spectra.append(spec)
                    headers.append(header)
                    filenames_list.append(filename)
                    logn_values.append(logn_value)
                    tex_values.append(tex_value)
                except ValueError:
                    print(f"Error converting data in: {filename}")
    
    if not spectra:
        raise ValueError("No valid data found in the folder.")
    
    min_freq, max_freq = min(map(np.min, frequencies)), max(map(np.max, frequencies))
    new_x = np.linspace(min_freq, max_freq, interp_points)
    interpolated_spectra = [np.interp(new_x, freq, spec) for freq, spec in zip(frequencies, spectra)]
    
    print(f"Data loaded successfully from {folder}. Data shape: {len(interpolated_spectra)}")
    return np.tile(new_x, (len(interpolated_spectra), 1)), np.array(interpolated_spectra), headers, filenames_list, np.array(logn_values), np.array(tex_values)

# 2. CNN MODEL BUILDING
def build_cnn_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=input_shape),
        layers.Conv1D(32, kernel_size=5, activation='relu', padding='same'),
        layers.Conv1D(64, kernel_size=5, activation='relu', padding='same'),
        layers.MaxPooling1D(2, padding='same') if input_shape[0] > 2 else layers.Activation('relu'),
        layers.Conv1D(128, kernel_size=5, activation='relu', padding='same'),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(input_shape[0], activation='linear')
    ])
    
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# 3. MODEL TRAINING
def train_and_evaluate_cnn(model, train_data):
    model.fit(train_data, train_data, epochs=50, batch_size=32, validation_split=0.1)
    return model

# 4. PLOTTING FUNCTIONS
def plot_logn_vs_predicted(test_logn, train_logn, closest_matches, test_tex, database_folder, pdf):
    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter(test_logn, train_logn[closest_matches], c=test_tex, cmap='viridis', alpha=0.7, edgecolors='k')
    plt.colorbar(scatter, label='Tex (K)')
    ax.plot([min(test_logn), max(test_logn)], [min(test_logn), max(test_logn)], 'r--', label='Ideal Prediction')
    ax.set_xlabel("True LogN (cm²)")
    ax.set_ylabel("Predicted LogN (cm²)")
    ax.set_title(f"Predicted vs True LogN - {os.path.basename(database_folder)}")
    ax.legend()
    ax.grid()
    pdf.savefig(fig)
    plt.close(fig)

def plot_tex_vs_predicted(test_tex, train_tex, closest_matches, test_logn, database_folder, pdf):
    fig, ax = plt.subplots(figsize=(8, 6))
    scatter = ax.scatter(test_tex, train_tex[closest_matches], c=test_logn, cmap='plasma', alpha=0.7, edgecolors='k')
    plt.colorbar(scatter, label='LogN (cm²)')
    ax.plot([min(test_tex), max(test_tex)], [min(test_tex), max(test_tex)], 'r--', label='Ideal Prediction')
    ax.set_xlabel("True Tex (K)")
    ax.set_ylabel("Predicted Tex (K)")
    ax.set_title(f"Predicted vs True Tex - {os.path.basename(database_folder)}")
    ax.legend()
    ax.grid()
    pdf.savefig(fig)
    plt.close(fig)

def plot_spectra(test_data, test_freq, train_data, train_freq, closest_matches, test_filenames, test_headers, train_filenames, train_headers, reconstructed_test, pdf):
    n_samples = min(10, len(test_data))
    indices = np.random.choice(len(test_data), n_samples, replace=False)
    
    fig = plt.figure(figsize=(15, 30))
    for i, idx in enumerate(indices):
        best_match_idx = closest_matches[idx]
        ax = fig.add_subplot(5, 2, i+1)
        ax.plot(test_freq[idx], test_data[idx], label='Test Spectrum', color='blue', linewidth=2)
        ax.plot(train_freq[best_match_idx], train_data[best_match_idx], label='Closest Match', linestyle='dashed', color='green', linewidth=2)
        ax.plot(test_freq[idx], reconstructed_test[idx], label='Reconstructed (CNN)', linestyle='dotted', color='orange', linewidth=2)
        ax.legend()
        ax.set_xlabel("Frequency (Hz)")
        ax.set_ylabel("Intensity")
        ax.set_title(f"Test {i+1} vs Match {best_match_idx}")
        ax.text(0.5, -0.3, f"Test File: {test_filenames[idx]}\nHeader: {test_headers[idx]}\n\n" 
                 f"Match: {train_filenames[best_match_idx]}\nHeader: {train_headers[best_match_idx]}",
                 fontsize=10, ha='center', va='top', transform=ax.transAxes)
    
    fig.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

# 5. INPUT SPECTRUM PROCESSING
def process_input_spectrum(filepath, model, train_data, encoded_train, train_freq, train_filenames, train_headers, pdf):
    try:
        print(f"\nProcessing input spectrum: {filepath}")
        
        if not os.path.exists(filepath):
            raise FileNotFoundError(f"Input spectrum file not found: {filepath}")
        
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            header = lines[0].strip() if len(lines) > 0 else ""
            
            data = []
            for line in lines[1:]:
                line = line.strip()
                if line and not line.startswith("//"):
                    parts = re.split(r'[\t ]+', line)
                    if len(parts) == 2:
                        try:
                            freq_ghz = float(parts[0])
                            temp_k = float(parts[1])
                            data.append((freq_ghz * 1e9, temp_k))
                        except ValueError:
                            continue
            
            if not data:
                raise ValueError("No valid data found in input spectrum file")
            
            freq, spec = zip(*data)
            freq = np.array(freq)
            spec = np.array(spec)

        train_min_freq = np.min(train_freq)
        train_max_freq = np.max(train_freq)
        new_x = np.linspace(train_min_freq, train_max_freq, 1000)
        interpolated_spec = np.interp(new_x, freq, spec, left=0, right=0)
        
        train_min = np.min(train_data)
        train_max = np.max(train_data)
        normalized_spec = (interpolated_spec - train_min) / (train_max - train_min + 1e-10)
        
        input_spec = np.expand_dims(normalized_spec, axis=[0, -1])
        encoded_spec = model.predict(input_spec, verbose=0)
        distances = euclidean_distances(encoded_spec.reshape(1, -1), 
                                      encoded_train.reshape(len(encoded_train), -1))
        closest_idx = np.argmin(distances[0])
        closest_dist = distances[0][closest_idx]
        
        reconstructed = model.predict(input_spec, verbose=0)
        reconstructed = reconstructed[0,:,0] if reconstructed.ndim == 3 else reconstructed[0]

        fig = plt.figure(figsize=(16, 12))
        gs = plt.GridSpec(2, 1, height_ratios=[3, 1], hspace=0.4)
        
        ax = fig.add_subplot(gs[0])
        all_curves = np.concatenate([normalized_spec, 
                                   train_data[closest_idx,:,0], 
                                   reconstructed])
        y_min = np.min(all_curves) - 0.1 * np.ptp(all_curves)
        y_max = np.max(all_curves) + 0.1 * np.ptp(all_curves)
        
        ax.plot(new_x, normalized_spec, 
               label='Input Spectrum (Normalized)', 
               color='navy', linewidth=2, alpha=0.5)
        ax.plot(train_freq[closest_idx], train_data[closest_idx,:,0], 
               label=f'Best Match (dist={closest_dist:.4f})', 
               linestyle='--', color='darkgreen', linewidth=2, alpha=0.5)
        ax.plot(new_x, reconstructed, 
               label='CNN Reconstruction', 
               linestyle=':', color='crimson', linewidth=2.5, alpha=0.5)
        
        ax.set_ylim(y_min, y_max)
        ax.set_title(f"Input Spectrum Analysis: {os.path.basename(filepath)}", pad=15, fontsize=14)
        ax.set_xlabel("Frequency (Hz)", fontsize=12)
        ax.set_ylabel("Normalized Intensity", fontsize=12)
        ax.grid(True, alpha=0.2)
        ax.legend(fontsize=10, loc='upper right')
        
        text_ax = fig.add_subplot(gs[1])
        text_ax.axis('off')
        
        match_info = (
            f"INPUT FILE:\n"
            f"{'='*50}\n"
            f"Filename: {os.path.basename(filepath)}\n"
            f"Header: {header[:200]}\n\n"
            f"BEST MATCH:\n"
            f"{'='*50}\n"
            f"• File: {train_filenames[closest_idx]}\n"
            f"• Distance: {closest_dist:.6f}\n"
            f"• Parameters: {train_headers[closest_idx][:200]}"
        )
        
        text_ax.text(0.5, 0.5, match_info, 
                   ha='center', va='center', fontsize=9, 
                   bbox=dict(facecolor='whitesmoke', alpha=0.7, boxstyle='round', pad=0.5),
                   fontfamily='monospace',
                   transform=text_ax.transAxes,
                   linespacing=1.8)
        
        plt.tight_layout()
        pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)
        plt.close(fig)
        
    except Exception as e:
        print(f"Error processing input spectrum: {str(e)}")
        traceback.print_exc()

# 6. MAIN EXECUTION
def main():
    directories = [
        r"C:\1. AI - ITACA\9. JupyterNotebook\JPL_CH3CN",
        r"C:\1. AI - ITACA\9. JupyterNotebook\JPL_CO",
        r"C:\1. AI - ITACA\9. JupyterNotebook\JPL_HCO+v=0,1,2",
        r"C:\1. AI - ITACA\9. JupyterNotebook\JPL_SiO",
        r"C:\1. AI - ITACA\9. JupyterNotebook\ALL"
    ]

    input_spectrum_file = r"C:\1. AI - ITACA\9. JupyterNotebook\_BK\example_4mols_format"

    for database_folder in directories:
        pdf_filename = f"CNN_PERFORMANCE_+_INPUTSPECCLASS_{os.path.basename(database_folder)}.pdf"
        
        with PdfPages(pdf_filename) as pdf:
            try:
                print(f"\nProcessing: {database_folder}")
                frequencies, data, headers, filenames, logn_values, tex_values = load_data(database_folder, interp_points=1000)
                data = (data - np.min(data)) / (np.max(data) - np.min(data))
                
                train_data, test_data, train_freq, test_freq, train_headers, test_headers, \
                train_filenames, test_filenames, train_logn, test_logn, train_tex, test_tex = train_test_split(
                    data, frequencies, headers, filenames, logn_values, tex_values, 
                    test_size=0.2, random_state=42, shuffle=True
                )
                
                train_data, test_data = np.expand_dims(train_data, axis=-1), np.expand_dims(test_data, axis=-1)
                
                model = build_cnn_model((train_data.shape[1], 1))
                model = train_and_evaluate_cnn(model, train_data)
                
                encoded_test = model.predict(test_data)
                encoded_train = model.predict(train_data)
                closest_matches = np.argmin(euclidean_distances(encoded_test.reshape(len(encoded_test), -1), 
                                                 encoded_train.reshape(len(encoded_train), -1)), axis=1)
                
                plot_logn_vs_predicted(test_logn, train_logn, closest_matches, test_tex, database_folder, pdf)
                plot_tex_vs_predicted(test_tex, train_tex, closest_matches, test_logn, database_folder, pdf)
                
                reconstructed_test = model.predict(test_data)
                plot_spectra(test_data, test_freq, train_data, train_freq, closest_matches, 
                            test_filenames, test_headers, train_filenames, train_headers, 
                            reconstructed_test, pdf)
                
                if os.path.exists(input_spectrum_file):
                    process_input_spectrum(input_spectrum_file, model, train_data, encoded_train, 
                                       train_freq, train_filenames, train_headers, pdf)
                else:
                    print(f"Input spectrum file not found at: {input_spectrum_file}")
                
                print(f"\nAll results saved to {pdf_filename}")
                
            except Exception as e:
                print(f"Error processing {database_folder}: {e}")
                traceback.print_exc()

if __name__ == "__main__":
    main()


Processing: C:\1. AI - ITACA\9. JupyterNotebook\JPL_CH3CN
Data loaded successfully from C:\1. AI - ITACA\9. JupyterNotebook\JPL_CH3CN. Data shape: 936
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - loss: 5.2981e-04 - mae: 0.0030 - val_loss: 4.3276e-04 - val_mae: 0.0082
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 2.4416e-04 - mae: 0.0080 - val_loss: 1.2736e-04 - val_mae: 0.0048
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - loss: 7.5838e-05 - mae: 0.0032 - val_loss: 7.5101e-05 - val_mae: 0.0027
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 4.9526e-05 - mae: 0.0021 - val_loss: 6.0856e-05 - val_mae: 0.0023
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 5.0332e-05 - mae: 0.0020 - val_loss: 6.4003e-05 - val_mae: 0.0027
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 5.4828e-05 - mae: 0.0024 - val_loss: 5.8651e-05 - val_mae: 0.0029
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 4.1825e-05 - mae: 0.0022 - val_loss: 2.

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:228: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:229: UserWarning: Glyph 9 (	) missing from font(s) DejaVu Sans Mono.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)



All results saved to CNN_PERFORMANCE_+_INPUTSPECCLASS_JPL_CH3CN.pdf

Processing: C:\1. AI - ITACA\9. JupyterNotebook\JPL_CO
Data loaded successfully from C:\1. AI - ITACA\9. JupyterNotebook\JPL_CO. Data shape: 936
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - loss: 1.9580e-05 - mae: 2.4039e-04 - val_loss: 2.3774e-05 - val_mae: 3.8616e-04
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 1.6592e-05 - mae: 4.3809e-04 - val_loss: 2.2409e-05 - val_mae: 6.9213e-04
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 1.6031e-05 - mae: 6.9122e-04 - val_loss: 2.0812e-05 - val_mae: 8.0654e-04
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 1.5233e-05 - mae: 8.2793e-04 - val_loss: 1.7465e-05 - val_mae: 9.3434e-04
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 1.1336e-05 - mae: 9.4337e-04 - val_loss: 8.9551e-06 - val_mae: 0.0011
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 3.9556e-06 - mae: 9.0388e-04 - val_loss: 9.2332e-07 - val_mae: 5.

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:228: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:229: UserWarning: Glyph 9 (	) missing from font(s) DejaVu Sans Mono.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)



All results saved to CNN_PERFORMANCE_+_INPUTSPECCLASS_JPL_CO.pdf

Processing: C:\1. AI - ITACA\9. JupyterNotebook\JPL_HCO+v=0,1,2
Data loaded successfully from C:\1. AI - ITACA\9. JupyterNotebook\JPL_HCO+v=0,1,2. Data shape: 936
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - loss: 1.1448e-04 - mae: 0.0010 - val_loss: 1.2594e-04 - val_mae: 0.0023
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 7.4149e-05 - mae: 0.0022 - val_loss: 6.3722e-05 - val_mae: 0.0034
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 2.5929e-05 - mae: 0.0022 - val_loss: 3.8106e-06 - val_mae: 0.0013
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 2.9367e-06 - mae: 9.3119e-04 - val_loss: 8.2637e-07 - val_mae: 4.3061e-04
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 1.0772e-06 - mae: 3.7154e-04 - val_loss: 4.0181e-07 - val_mae: 2.5938e-04
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - loss: 8.6874e-07 - mae: 2.2063e-04 - val_loss: 4.3447e-07 - val_mae: 2.3069e

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:228: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:229: UserWarning: Glyph 9 (	) missing from font(s) DejaVu Sans Mono.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)



All results saved to CNN_PERFORMANCE_+_INPUTSPECCLASS_JPL_HCO+v=0,1,2.pdf

Processing: C:\1. AI - ITACA\9. JupyterNotebook\JPL_SiO
Data loaded successfully from C:\1. AI - ITACA\9. JupyterNotebook\JPL_SiO. Data shape: 936
Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - loss: 2.1464e-05 - mae: 2.0140e-04 - val_loss: 1.7122e-05 - val_mae: 3.4128e-04
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 1.3070e-05 - mae: 3.0164e-04 - val_loss: 1.6510e-05 - val_mae: 4.4051e-04
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - loss: 1.5340e-05 - mae: 4.6208e-04 - val_loss: 1.5733e-05 - val_mae: 5.4248e-04
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 1.4723e-05 - mae: 5.7290e-04 - val_loss: 1.4484e-05 - val_mae: 6.2597e-04
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 1.9192e-05 - mae: 8.1064e-04 - val_loss: 1.2597e-05 - val_mae: 6.2911e-04
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 1.1951e-05 - mae: 7.7998e-04 - val_loss: 6.7524e-06 -

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:228: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:229: UserWarning: Glyph 9 (	) missing from font(s) DejaVu Sans Mono.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)



All results saved to CNN_PERFORMANCE_+_INPUTSPECCLASS_JPL_SiO.pdf

Processing: C:\1. AI - ITACA\9. JupyterNotebook\ALL
Data loaded successfully from C:\1. AI - ITACA\9. JupyterNotebook\ALL. Data shape: 3744
Epoch 1/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 6s 53ms/step - loss: 1.7148e-04 - mae: 0.0018 - val_loss: 3.8090e-05 - val_mae: 0.0018
Epoch 2/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 2.3931e-05 - mae: 0.0013 - val_loss: 1.5164e-05 - val_mae: 9.9629e-04
Epoch 3/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 5s 55ms/step - loss: 1.5029e-05 - mae: 9.2031e-04 - val_loss: 7.3655e-06 - val_mae: 6.0495e-04
Epoch 4/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - loss: 8.9750e-06 - mae: 6.9163e-04 - val_loss: 9.1027e-06 - val_mae: 9.5129e-04
Epoch 5/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 1.2965e-05 - mae: 0.0011 - val_loss: 6.0050e-06 - val_mae: 6.1408e-04
Epoch 6/50
85/85 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - loss: 7.4688e-06 - mae: 6.5975e-04 - val_loss: 1.0670e-05 - val_mae: 6.4149e-04
Epoch 7/50

C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:228: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\Usuario\AppData\Local\Temp\ipykernel_17548\3680624584.py:229: UserWarning: Glyph 9 (	) missing from font(s) DejaVu Sans Mono.
  pdf.savefig(fig, bbox_inches='tight', pad_inches=0.5, dpi=300)
